In [14]:
from config import MAP_HEIGHT, MAP_SYMBOL, MAP_WIDTH

In [22]:
class Person:
        
    def __init__(self, x:int, y:int, char:str, health: int, attack: int, defense: int):
        self.x = x
        self.y = y
        self.char = char
        self.health = health
        self.attack = attack
        self.defense = defense
        self.is_alive = True

    def take_damage(self, damage:int):
        actual_damage = max(1, damage)
        self.health -= actual_damage
        if self.health < 0:
            self.health = 0
            self.is_alive = False
        return actual_damage


In [23]:
class Player(Person):

    def __init__(self, x:int, y:int):
        super().__init__(x=x, y = y, char = 'P', health = 50, attack = 10, defense=5)

    def change_place(self, direction:str):
        if direction == 'w' and self.y > 1:
            self.y -= 1
            moved=True
        elif direction == 's' and self.y < MAP_HEIGHT:
            self.y += 1
            moved=True
        elif direction == 'a' and self.x > 1:
            self.x -= 1
            moved=True
        elif direction== 'd' and self.x < MAP_WIDTH:
            self.x += 1
            moved=True
        else:
            print("Движение невозможно — выход за границы карты.")
            return False
        return True

In [24]:
class Enemy(Person):
    def __init__(self, x: int, y: int):
        super().__init__(x=x, y = y, char = 'E', health = 10, attack = 10, defense=5)


In [29]:
class Fight():
    def __init__(self, player: Player, enemy: Enemy):
        self.player = player
        self.enemy = enemy
    
    def calculate_damage(self, attacker, defender):
        damage = attacker.attack * (1 - defender.defense / 100)
        return max(1, int(damage))  # минимум 1 урон
    
    def start(self, verbose=True):
        if verbose:
            print('Бой начался!')
        
        while self.player.is_alive and self.enemy.is_alive:
        # Игрок атакует врага
            dmg = self.calculate_damage(self.player, self.enemy)
            self.enemy.take_damage(dmg)
            if verbose:
                print(f'Персонаж бьёт с атакой {self.player.attack} против защиты {self.enemy.defense}, наносит {dmg} урона')
                
                if self.enemy.is_alive:
                    print(f'У противника осталось {self.enemy.health} хп')
                else:
                    print('Вы победили!')
            if not self.enemy.is_alive:
                return True

        # Враг атакует игрока
            dmg = self.calculate_damage(self.enemy, self.player)
            self.player.take_damage(dmg)
            
            if verbose:
                print(f'Противник бьёт с атакой {self.enemy.attack} против защиты {self.player.defense}, наносит {dmg} урона')
                if self.player.is_alive:
                    print(f'У персонажа осталось {self.player.health} хп')
                else:
                    print('Вы проиграли!')
            
            if not self.player.is_alive:
                return False  # Игрок проиграл

In [30]:
def draw(player: Player, enemies: list):
    if  player.x > MAP_WIDTH or player.x <= 0:
        print('Указана неверная координата X у героя')
        return
    if player.y > MAP_HEIGHT or player.y<= 0:
        print('Указана неверная координата Y у героя')
        return

    for enemy in enemies:
        if enemy.x <= 0 or enemy.x > MAP_WIDTH or enemy.y <= 0 or enemy.y > MAP_HEIGHT:
            print(f'Ошибка: враг на ({enemy.x}, {enemy.y}) вне карты.')
            return

    for i in range(1, MAP_HEIGHT + 1):
        row = ''
        for j in range(1, MAP_WIDTH + 1):
            cell = MAP_SYMBOL
            if j == player.x and i == player.y:
                cell = player.char
            else:
                for enemy in enemies:
                    if j == enemy.x and i == enemy.y:
                        cell = 'e'
                        break
            row += cell
        print(row)


In [31]:
player = Player(2,3)
enemy1 = Enemy(2,4)
enemy2 = Enemy(1,5)
enemies = [enemy1, enemy2]

In [ ]:
print("Управление: W (вверх), A (влево), S (вниз), D (вправо), Q (выход)")
draw(player, enemies)

# Игра
while True:
    user_input = input("Введите команду: ").strip().lower()

    if user_input not in ['w', 'a', 's', 'd', 'q']:
        print("Неверная команда. Допустимые: W, A, S, D, Q.")
        continue

    if user_input == 'q':
        print("Игра окончена.")
        break

    moved = player.change_place(user_input)
    
    if moved:
        for enemy in enemies:
            if enemy.is_alive and player.x == enemy.x and player.y == enemy.y:
                print('Столкновение!')
                battle = Fight(player, enemy)
                result = battle.start()
                if not player.is_alive:
                    print("Игра окончена: вы погибли.")
                    exit()
                if not enemy.is_alive:
                    enemies.remove(enemy)  
                break

        print()  # пустая строка
        draw(player, enemies)



Управление: W (вверх), A (влево), S (вниз), D (вправо), Q (выход)
*****
*****
*P***
*e***
e****
Столкновение!
Бой начался!
Персонаж бьёт с атакой 10 против защиты 5, наносит 9 урона
У противника осталось 1 хп
Противник бьёт с атакой 10 против защиты 5, наносит 9 урона
У персонажа осталось 41 хп
Персонаж бьёт с атакой 10 против защиты 5, наносит 9 урона
Вы победили!

*****
*****
*****
*P***
e****

*****
*****
*****
P****
e****
Столкновение!
Бой начался!
Персонаж бьёт с атакой 10 против защиты 5, наносит 9 урона
У противника осталось 1 хп
Противник бьёт с атакой 10 против защиты 5, наносит 9 урона
У персонажа осталось 32 хп
Персонаж бьёт с атакой 10 против защиты 5, наносит 9 урона
Вы победили!

*****
*****
*****
*****
P****
Игра окончена.
